<a href="https://colab.research.google.com/github/kridtapon/ATR-Overextension-Strategy/blob/main/ATR_Overextension_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Average True Range (ATR)
def calculate_atr(df, period=14):
    """
    Calculate Average True Range (ATR).
    """
    df['HL'] = df['High'] - df['Low']
    df['HC'] = abs(df['High'] - df['Close'].shift())
    df['LC'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['HL', 'HC', 'LC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=period).mean()
    return df['ATR']

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate ATR
df['ATR'] = calculate_atr(df, period=14)

# Define ATR Overextension multiplier (e.g., 2x ATR for overextension)
atr_multiplier = 2

# Calculate Entry and Exit signals based on ATR Overextension
df['Entry'] = df['Close'] > df['Close'].shift(10) + atr_multiplier * df['ATR']
df['Exit'] = df['Close'] < df['Close'].shift(10) - atr_multiplier * df['ATR']

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           335461.493406
Total Return [%]                       235.461493
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                       8196.588955
Max Drawdown [%]                        37.159804
Max Drawdown Duration                       612.0
Total Trades                                   22
Total Closed Trades                            22
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            40.909091
Best Trade [%]                          62.565813
Worst Trade [%]                         -9.890704
Avg Winning Trade [%]                   24.374172
Avg Losing Trade [%]                    -4.302001


In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from itertools import product

# Function to calculate Average True Range (ATR)
def calculate_atr(df, period=14):
    df['HL'] = df['High'] - df['Low']
    df['HC'] = abs(df['High'] - df['Close'].shift())
    df['LC'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['HL', 'HC', 'LC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=period).mean()
    return df['ATR']

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Define optimization parameter ranges
atr_period_range = range(5, 21)  # ATR periods from 5 to 20
atr_multiplier_range = np.arange(1.0, 3.1, 0.5)  # ATR multiplier from 1.0 to 3.0
shift_range = range(1, 21)  # Shift value for the closing price comparison

# Generate all combinations of the parameters
param_combinations = product(atr_period_range, atr_multiplier_range, shift_range)

# Initialize variables to store the best results
best_return = -np.inf
best_params = None
best_portfolio = None

# Loop through all combinations of parameters
for atr_period, atr_multiplier, shift_val in param_combinations:
    # Calculate ATR with the current period
    df['ATR'] = calculate_atr(df, period=atr_period)

    # Create Entry and Exit signals based on ATR and shift
    df['Entry'] = df['Close'] > df['Close'].shift(shift_val) + atr_multiplier * df['ATR']
    df['Exit'] = df['Close'] < df['Close'].shift(shift_val) - atr_multiplier * df['ATR']

    # Filter data for the test period (2020-2025)
    df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_filtered['Close'],
        entries=df_filtered['Entry'],
        exits=df_filtered['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    # Evaluate performance using Total Return as the metric
    total_return = portfolio.stats()['Total Return [%]']

    # If the performance is better, store the current parameters and portfolio
    if total_return > best_return:
        best_return = total_return
        best_params = (atr_period, atr_multiplier, shift_val)
        best_portfolio = portfolio

# Display the best parameters and portfolio performance
print(f"Best Parameters: ATR Period = {best_params[0]}, ATR Multiplier = {best_params[1]}, Shift = {best_params[2]}")
print(best_portfolio.stats())

# Plot equity curve for the best portfolio
best_portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: 

Best Parameters: ATR Period = 11, ATR Multiplier = 2.5, Shift = 2
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           738208.630201
Total Return [%]                        638.20863
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                        402.747758
Max Drawdown [%]                        23.275806
Max Drawdown Duration                       349.0
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                      586784.952749
Win Rate [%]                                100.0
Best Trade [%]                          51.475101
Worst Trade [%]                         51.475101
Avg Winning Trade [%]             